In [ ]:
# @title 1. Group Segments (Heavy vs Light)

# CELL 1 [TAG: parameters]

# Default parameters (Airflow will inject these)

INPUT_FOLDER = "s3://processed-data/"       # Output from Step 0

OUTPUT_GROUPED_DATA = "s3://models/grouped_segments.pkl"
SPEED_THRESHOLD = 25.0  # km/h
MIN_DURATION = 15       # seconds

MINIO_ENDPOINT = "http://localhost:9000" 
MINIO_ACCESS_KEY = "admin"
MINIO_SECRET_KEY = "password123"

In [ ]:
# CELL 2: Imports

import os

import pandas as pd

import numpy as np

import pickle

import s3fs


In [ ]:
# CELL 3: MinIO Configuration

# Define connection settings for S3-compatible storage

MINIO_ENDPOINT = os.environ.get("MINIO_ENDPOINT", "http://minio:9000")

MINIO_KEY = os.environ.get("MINIO_ACCESS_KEY", "admin")

MINIO_SECRET = os.environ.get("MINIO_SECRET_KEY", "password123")

# Initialize S3 Filesystem

fs = s3fs.S3FileSystem(
    key=MINIO_ACCESS_KEY,      # Uses the variable injected by Papermill
    secret=MINIO_SECRET_KEY,   # Uses the variable injected by Papermill
    client_kwargs={'endpoint_url': MINIO_ENDPOINT}
)

# Pandas storage options
storage_options = {
    "key": MINIO_ACCESS_KEY,
    "secret": MINIO_SECRET_KEY,
    "client_kwargs": {"endpoint_url": MINIO_ENDPOINT}
}


In [ ]:
# CELL 4: Logic

def preprocess_and_group(folder):

    grouped_segments = [[], []] # Index 0: Heavy, Index 1: Light

    # 1. List all processed 1Hz files in the S3 bucket

    # fs.glob returns paths like 'bucket/file.csv', we need to ensure 's3://' prefix if needed

    # or just pass the path directly to fs.open

    try:

        file_paths = fs.glob(f"{folder.replace('s3://', '')}*.csv")

    except Exception as e:

        print(f"Error listing files in {folder}: {e}")

        return grouped_segments

    print(f"Found {len(file_paths)} processed trips.")

    for file_path in file_paths:

        print(f"Processing {file_path}...")

        

        # Read directly from S3

        with fs.open(file_path, 'rb') as f:

            df = pd.read_csv(f)

        

        # Ensure sorted by time

        df = df.sort_values('seconds_elapsed')

        # Identify contiguous blocks of the same segment_id

        # (Step 0 already ensured segment_id is populated via Google Roads API)

        df['block_id'] = (df['segment_id'] != df['segment_id'].shift()).cumsum()

        for _, block in df.groupby('block_id'):

            # Filter out short noise segments

            if len(block) < MIN_DURATION: 

                continue

            # Extract Physics

            speed_kmh = block['speed_kmh'].values

            

            # CRITICAL CHANGE: Use the Sensor-Fused Acceleration from Step 0

            # Instead of recalculating it via derivative (np.diff), we use the 

            # high-precision 'acc_forward' calculated from the IMU.

            if 'acc_forward' in block.columns:

                accel_ms2 = block['acc_forward'].values

            else:

                # Fallback if column missing (shouldn't happen with Step 0)

                accel_ms2 = np.zeros(len(speed_kmh))

                accel_ms2[:-1] = np.diff(speed_kmh) / 3.6

            

            # Stack into matrix: [Speed, Acceleration]

            segment_data = np.column_stack((speed_kmh, accel_ms2))

            # Classify based on Average Speed

            if np.mean(speed_kmh) <= SPEED_THRESHOLD:

                grouped_segments[0].append(segment_data) # Heavy

            else:

                grouped_segments[1].append(segment_data) # Light

    return grouped_segments


In [ ]:
# CELL 5: Execution & Save

data = preprocess_and_group(INPUT_FOLDER)

print(f"Result: {len(data[0])} Heavy segments, {len(data[1])} Light segments.")

# Save Pickle to S3

print(f"Saving model data to {OUTPUT_GROUPED_DATA}...")

with fs.open(OUTPUT_GROUPED_DATA, 'wb') as f:

    pickle.dump(data, f)

print("✅ Done.")
